In [4]:
pip install pytube opencv-python mediapipe numpy

Note: you may need to restart the kernel to use updated packages.


In [7]:
import cv2
import mediapipe as mp
import numpy as np
import os
import json
import yt_dlp
import math
from collections import deque
import time


# 1) Configurations

VIDEO_URL = "https://www.youtube.com/shorts/vSX3IRxGnNY"
VIDEO_PATH = "input_video.mp4"
OUTPUT_DIR = "output"
OUTPUT_VIDEO_PATH = os.path.join(OUTPUT_DIR, "annotated_video.mp4")
EVALUATION_JSON = os.path.join(OUTPUT_DIR, "evaluation.json")

os.makedirs(OUTPUT_DIR, exist_ok=True)


# 2) Download YouTube video 

if not os.path.exists(VIDEO_PATH):
    print("[INFO] Downloading video from YouTube...")
    ydl_opts = {'format': 'best', 'outtmpl': VIDEO_PATH}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([VIDEO_URL])
    print("[INFO] Download complete!")


# 3) Setup MediaPipe 

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

pose = mp_pose.Pose(min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)


# 4) Thresholds

THRESH = {
    "elbow_good_min": 120.0,
    "elbow_good_max": 175.0,
    "spine_good_min": 5.0,
    "spine_good_max": 25.0,
    "head_knee_max_gap_frac": 0.06,
    "foot_angle_max_abs": 25.0,
    "visibility_thresh": 0.4,
    "ema_alpha": 0.18,
}


# 5) Helper functions

def calculate_angle(a, b, c):
    """Calculate angle at point b formed by points a-b-c in degrees."""
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba, bc = a-b, c-b
    cos_angle = np.dot(ba, bc) / (np.linalg.norm(ba)*np.linalg.norm(bc)+1e-8)
    return np.degrees(np.arccos(np.clip(cos_angle, -1.0, 1.0)))

def get_coords(landmarks, idx, width, height):
    """Get pixel coordinates from landmark index."""
    return [landmarks[idx].x * width, landmarks[idx].y * height]

def angle_between(v1, v2):
    """Angle between two vectors."""
    v1, v2 = np.array(v1), np.array(v2)
    norm1, norm2 = np.linalg.norm(v1), np.linalg.norm(v2)
    if norm1<1e-8 or norm2<1e-8: return 0
    cos = np.clip(np.dot(v1,v2)/(norm1*norm2), -1, 1)
    return math.degrees(math.acos(cos))

def safe_div(a,b):
    return a/b if b!=0 else 0

def score_from_ratio(ratio):
    """Convert 0-1 ratio to 1-10 score (nonlinear)."""
    ratio = max(0.0, min(1.0, ratio))
    return int(round(1 + 9*(ratio**0.85)))


# 6) Open video

cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise IOError("[ERROR] Cannot open video file.")

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = int(cap.get(cv2.CAP_PROP_FPS))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (width, height))


# 7) Frame processing variables

metrics_history = []
ema_state = {"elbow": None, "spine": None, "head_knee": None, "foot": None}
prev_elbow = None
alpha = THRESH["ema_alpha"]
frame_idx = 0
t_start = time.time()
elbow_history = deque(maxlen=100)  # for rolling plot

#
# 8) Main frame looop

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_idx += 1
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)
    display_frame = frame.copy()
    frame_metrics = {}

    if results.pose_landmarks:
        lms = results.pose_landmarks.landmark
        # ---- Keypoints ----
        l_shoulder = get_coords(lms, mp_pose.PoseLandmark.LEFT_SHOULDER.value, width, height)
        l_elbow = get_coords(lms, mp_pose.PoseLandmark.LEFT_ELBOW.value, width, height)
        l_wrist = get_coords(lms, mp_pose.PoseLandmark.LEFT_WRIST.value, width, height)
        l_hip = get_coords(lms, mp_pose.PoseLandmark.LEFT_HIP.value, width, height)
        l_knee = get_coords(lms, mp_pose.PoseLandmark.LEFT_KNEE.value, width, height)
        l_heel = get_coords(lms, mp_pose.PoseLandmark.LEFT_HEEL.value, width, height)
        l_foot = get_coords(lms, mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value, width, height)
        nose = get_coords(lms, mp_pose.PoseLandmark.NOSE.value, width, height)

        # ---- Matrics calculation ----
        elbow_angle = calculate_angle(l_shoulder,l_elbow,l_wrist)
        spine_lean = angle_between([l_shoulder[0]-l_hip[0], l_shoulder[1]-l_hip[1]], [0,-1])
        head_knee_gap_frac = safe_div(abs(nose[0]-l_knee[0]), width)
        foot_vec = [l_foot[0]-l_heel[0], l_foot[1]-l_heel[1]]
        foot_angle = angle_between(foot_vec,[1,0])
        cross_z = foot_vec[0]*0.0 - foot_vec[1]*1.0
        foot_angle = -foot_angle if cross_z<0 else foot_angle

        frame_metrics.update({"elbow": elbow_angle, "spine": spine_lean,
                              "head_knee": head_knee_gap_frac, "foot": foot_angle})

        # ---- EMA smoothing ----
        display_vals = {}
        for k,v in frame_metrics.items():
            if ema_state[k] is None:
                ema_state[k] = v
            else:
                ema_state[k] = alpha*v + (1-alpha)*ema_state[k]
            display_vals[k] = ema_state[k]

        # ---- Overlay metrics ----
        y0, dy = 30, 30
        cv2.putText(display_frame, f"Elbow: {display_vals['elbow']:.1f}°", (10,y0),
                    cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,0),2)
        cv2.putText(display_frame, f"Spine: {display_vals['spine']:.1f}°", (10,y0+dy),
                    cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,0),2)
        cv2.putText(display_frame, f"Head-Knee: {display_vals['head_knee']*100:.1f}%W", (10,y0+2*dy),
                    cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,0),2)
        cv2.putText(display_frame, f"Front Foot: {display_vals['foot']:.1f}°", (10,y0+3*dy),
                    cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,0),2)

        # ---- Short feedback ----
        fb = []
        fb.append("✅ Good elbow" if THRESH["elbow_good_min"]<elbow_angle<THRESH["elbow_good_max"] else "❌ Adjust elbow")
        fb.append("✅ Spine balanced" if THRESH["spine_good_min"]<spine_lean<THRESH["spine_good_max"] else "❌ Spine lean off")
        fb.append("✅ Head over knee" if head_knee_gap_frac<THRESH["head_knee_max_gap_frac"] else "❌ Head too far forward")
        fb.append("✅ Foot aligned" if abs(foot_angle)<THRESH["foot_angle_max_abs"] else "❌ Foot misaligned")

        for i,f in enumerate(fb):
            color = (0,200,0) if f.startswith("✅") else (0,0,255)
            cv2.putText(display_frame,f,(10,y0+5*dy+i*dy),cv2.FONT_HERSHEY_SIMPLEX,0.7,color,2)

      
        mp_drawing.draw_landmarks(display_frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=3),
                                  connection_drawing_spec=mp_drawing.DrawingSpec(color=(255,0,0), thickness=2))

  
        if prev_elbow is not None:
            frame_metrics['elbow_smooth'] = abs(elbow_angle-prev_elbow)
        prev_elbow = elbow_angle
        elbow_history.append(elbow_angle)

       
        plot_h, plot_w = 50, width
        plot_img = np.zeros((plot_h,plot_w,3), dtype=np.uint8)
        for idx,val in enumerate(elbow_history):
            x = int(idx*plot_w/len(elbow_history))
            y = int(plot_h - (val/180)*plot_h)
            cv2.circle(plot_img,(x,y),1,(0,255,255),-1)
        display_frame[-plot_h:,-plot_w:] = plot_img

    metrics_history.append(frame_metrics)

    # ---- FPS display ----
    elapsed = time.time()-t_start
    fps_est = frame_idx/elapsed if elapsed>0 else 0
    cv2.putText(display_frame,f"FPS: {fps_est:.1f}",(width-150,30),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,255,255),2)

    out.write(display_frame)
    cv2.imshow("Cricket Cover Drive Analysis", display_frame)
    if cv2.waitKey(1) & 0xFF==27:  # ESC to quit
        break


# 9) Release 

cap.release()
out.release()
pose.close()
cv2.destroyAllWindows()

# -----------------------
# 10) Aggregate scores
# -----------------------
n_frames = max(1,len(metrics_history))
elbow_ok = sum(1 for m in metrics_history if 'elbow' in m and THRESH["elbow_good_min"]<=m['elbow']<=THRESH["elbow_good_max"])
spine_ok = sum(1 for m in metrics_history if 'spine' in m and THRESH["spine_good_min"]<=m['spine']<=THRESH["spine_good_max"])
head_ok = sum(1 for m in metrics_history if 'head_knee' in m and m['head_knee']<THRESH["head_knee_max_gap_frac"])
foot_ok = sum(1 for m in metrics_history if 'foot' in m and abs(m['foot'])<THRESH["foot_angle_max_abs"])
elbow_diffs = [m.get('elbow_smooth',0) for m in metrics_history if 'elbow_smooth' in m]
smooth_factor = 1.0 - np.mean(elbow_diffs)/40.0 if elbow_diffs else 0.5
smooth_factor = min(max(smooth_factor,0),1)

scores = {
    "Footwork": score_from_ratio(foot_ok/n_frames),
    "Head Position": score_from_ratio(head_ok/n_frames),
    "Swing Control": score_from_ratio(smooth_factor),
    "Balance": score_from_ratio(0.5*spine_ok/n_frames + 0.5*head_ok/n_frames),
    "Follow-through": score_from_ratio(0.5*elbow_ok/n_frames + 0.5*spine_ok/n_frames),
}

comments = {
    "Footwork": "Good alignment when foot_angle within range." if foot_ok/n_frames>0.6 else "Work on front foot aiming; reduce toe-out.",
    "Head Position": "Head consistently over front knee." if head_ok/n_frames>0.6 else "Keep head stacked over front knee.",
    "Swing Control": "Smooth elbow motion." if np.mean(elbow_diffs)<12 else "Reduce abrupt elbow changes.",
    "Balance": "Stable posture and spine lean." if spine_ok/n_frames>0.6 else "Maintain slight forward lean (5–25°).",
    "Follow-through": "Good continuation of swing." if (0.5*elbow_ok/n_frames + 0.5*spine_ok/n_frames)>0.6 else "Complete follow-through with tall posture.",
}

evaluation = {
    "video": os.path.abspath(VIDEO_PATH),
    "frames": n_frames,
    "avg_fps": round(fps_est,2),
    "scores": scores,
    "comments": comments,
    "thresholds": THRESH
}

with open(EVALUATION_JSON,'w') as f:
    json.dump(evaluation,f,indent=2)

print("✅ Processing complete!")
print(f"Annotated video saved at: {OUTPUT_VIDEO_PATH}")
print(f"Evaluation JSON saved at: {EVALUATION_JSON}")


✅ Processing complete!
Annotated video saved at: output\annotated_video.mp4
Evaluation JSON saved at: output\evaluation.json
